<img src="img/ff-logo.png" width="30%">

## LDA workshop

### Mike Williams • Director of Research • Fast Forward Labs

#### [github.com/fastforwardlabs/ldaworkshop](https://github.com/fastforwardlabs/ldaworkshop)

# Text

There are lots of things you might want to do with documents
 - group them together
 - summarize one or several of them
 - classify them (e.g. sentiment, tagging)
 - explore, e.g. temporal trends in what they discuss

To do any of these things using a computer, you first need to convert words into numbers.

## Bags of words

The "bag of words" approach is the simplest method for this.

![Bag of words](img/bagwords.png)

There are at least three problems with this approach:

 - **long, sparse data**: a short text is turned into a very long series of numbers, most of which are zero ("sparse").
 
 - **synonyms and multiple meanings**: "movies" and "films" are different (they should not be), and "bow" (that shoots an arrow) and "bow" (you make to a King) are not (they should be)
 
 - **word order**: totally lost ("man bites dog" and "dog bites man" are the same).

We're going to look at a technique that addresses the first of these two problems.

# Topic modeling

Topic modeling is a statistical method to find groups of words that tend to co-occur in a corpus of documents.

For example, maybe the words "movie", "film" and "director" often occur in the same documents. That would make them a "topic".

Topic modeling algorithms find these groups automatically. They are an instance of the class of algorithms known as "unsupervised machine learning".

In doing this, we become able to express documents as a combination of a relatively small number (~100) of topics, rather than thousands of words (most of which don't occur), and we can treat documents about "films" and "movies" similarly.

# Topic modeling workflow

Topic modeling has two steps:

 - learn topics from a corpus of representative documents
 - figure out which of these topics occur in the particular document(s) you're interested in
 
If you're a machine learning person, you'll recognize these as training and evaluation.

<img src="img/lda_topics.png" width=50%>

<img src="img/lda_evaluate.png" width=50%>

Once you've done the second step, you've expressed your new document as a short vector of numbers that you can now do all sorts of things with:

 - group documents together
 - summarize one or several documents
 - classify it (e.g. sentiment, tagging)
 - explore, e.g. temporal trends in what people are discussing, e.g. [Time-series plots of 1000 topics extracted from 20 years of the New York Times.](http://christo.cs.umass.edu/NYT/)

## Latent Dirichlet Allocation

The best known and best algorithm for finding topics in a corpus is Latent Dirichlet Allocation. It's got a complicated name and, to be frank, it's a complicated algorithm. If you'd like to begin to dig into the details, there are two resources I recommend very highly!

 - [Tim Hopper's PyData NYC 2015 talk](https://www.youtube.com/watch?v=_R66X_udxZQ)
 - [David Blei's ACM article](https://www.cs.princeton.edu/~blei/papers/Blei2012.pdf)
 
For the purposes of this talk, I'm just going to say it finds groups of words that co-occur by magic!

The good news is, there are several excellent open source implementations of the algorithm, and we're going to use one of those today.

We're going to apply it to a public dataset of Amazon product reviews.

# Load data

The cell below opens the file and loads it into a pandas dataframe.

There's a lot going on in this line, all of which is useful to understand if you're a Python programmer, but none of which is necessary to understand if you're only interested in LDA.

If you would like a more detailed explanation of what's going on, please see [the Data notebook](data.ipynb)

In [1]:
import json
import numpy as np
import pandas as pd
from __future__ import print_function

with open("reviews.json", 'rb') as f:
    reviews = pd.DataFrame(json.loads(l.decode()) for l in f)

A pandas dataframe is a structured table-like object that, among many other things, supports a bunch of SQL-like operations and handles fiddly data types like data and times well. I don't know much about R, but I understand R has objects like this too.

`head` allows us to see the first few rows:

In [2]:
reviews.head(2)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1223000893,"[0, 0]",3.0,I purchased the Trilogy with hoping my two cat...,"01 12, 2011",A14CK12J7C7JRK,Consumer in NorCal,Nice Distraction for my cats for about 15 minutes,1294790400
1,1223000893,"[0, 0]",5.0,There are usually one or more of my cats watch...,"09 14, 2013",A39QHP5WLON5HV,Melodee Placial,Entertaining for my cats,1379116800


Individual columns can be accessed as keys:

In [3]:
reviews['asin'].head()

0    1223000893
1    1223000893
2    1223000893
3    1223000893
4    1223000893
Name: asin, dtype: object

In [4]:
print("{} reviews".format(len(reviews)))
print("of {} products ".format(len(reviews.asin.unique())))
print("by {} unique authors ".format(len(reviews.reviewerID.unique())))

150000 reviews
of 8019 products 
by 19834 unique authors 


In [5]:
texts = reviews.loc[:19999, 'reviewText']

In [6]:
print(texts[0])

I purchased the Trilogy with hoping my two cats, age 3 and 5 would be interested.  The 3 yr old cat was fascinated for about 15 minutes but when the same pictures came on, she got bored.  The 5 year old watched for about a few minutes but then walked away. It is possible that because we have a wonderful courtyard full of greenery and trees and one of my neighbors has a bird feeder, that there is enough going on outside that they prefer real life versus a taped version.  I will more than likely pass this on to a friend who has cats that don't have as much wildlife to watch as mine do.


# Vectorize reviews

You have to preprocess the text a little before you apply LDA: you need to split documents into words, and you need to turn words into vectorized numbers.

Ironically, in order to get the benefits of LDA, we first need to run bag of words on our data!

The code to do this is built into scikit-learn.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
vectorizer.fit(texts)
X = vectorizer.transform(texts)

In [8]:
X.shape

(20000, 10000)

# Learn topics using LDA

In [9]:
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
%%time
lda = LatentDirichletAllocation(n_topics=100, learning_method='batch', n_jobs=-2, random_state=0)
lda.fit(X)

CPU times: user 2.3 s, sys: 4.8 s, total: 7.1 s
Wall time: 2min


# Inspect topics

The first thing you should do when you fit a topic model is inspect a few of the words that dominate each topic to check that the topics are coherent.

To do this, we need to look at the `components_` attribute now attached to `lda`. This is an array with `n_topics` rows and a number of columns equal to the size of the vocabulary.

In [11]:
print(lda.components_.shape)

(100, 10000)


Each number in this array is the weight of the corresponding word in the corresponding topic.

The weights of each topic should add up to one, i.e. each row of `lda.components_` should add up to 1.

In [12]:
lda.components_.sum(axis=1)

array([  4534.79197819,   6536.33606203,   8059.76906425,   4799.81864673,
         5018.78957358,   5301.49111286,   4448.68859461,   4580.69012866,
         5474.41128545,   9256.079117  ,   6762.49331627,   7415.62152883,
         4418.81854245,   3980.58060342,   7350.43200465,   5773.15005322,
         3519.23521742,   6282.42644941,   5190.89178647,   5198.83203309,
         2904.51328497,   3666.94173982,   9781.3688993 ,  12388.86004121,
         4562.30544647,   3341.41537675,   3787.12776976,   3867.49493259,
         3703.91091669,  15538.35449758,   9066.05154524,   4185.55160085,
        12752.7603493 ,   4415.27458863,   8593.95057063,   8355.05174169,
         5146.18221575,   4021.15133261,  14239.45677745,   5679.67622282,
         5764.66465695,   4669.28425848,  36735.94415792,   3065.2023081 ,
         4845.76216147,  11269.84932744,   5213.97881403,   4806.57905922,
         8836.07796354,   8285.49544316,   3791.11078225,  15068.73552095,
         6226.32274077,  

Oh dear! It turns out there's a bug in scikit-learn's implementation of LDA. Let's fix it here. This should be fixed in the next version of scikit-learn.

In [13]:
lda.components_ /= lda.components_.sum(axis=1)[:, None]

This array of topics and words (or terms) is usually called the topic-term matrix, so let's save it under that name:

In [14]:
topic_term = lda.components_

The word corresponding to each column in this array, which we'll call the `vocabulary`, is available as a list from `vectorizer.get_feature_names()`

In [15]:
vocabulary = vectorizer.get_feature_names()
print(vocabulary[:10000:1000]) # print the 0th, 999th, 1999th, 2999th, etc. item in the vocabulary

['00', 'bettas', 'contemplating', 'elegant', 'grunter', 'lays', 'nutro', 'ranchus', 'skinnier', 'tipped']


Now let's look at the top 10 words that dominate each topic.

It does that by going through each row (i.e. each topic) in the `topic_term` array, finding the biggest numbers in that row, then finding the corresponding word.

In [16]:
def print_topic(topic_term, topic_id, vocabulary):
    print("{:2d} ".format(topic_id) + " ".join(vocabulary[i] for i in topic_term[topic_id].argsort()[:-11:-1]))

for i, _  in enumerate(topic_term):
    print_topic(topic_term, i, vocabulary)

 0 bowl water small large happy clean time size does hamster
 1 bed warm heat night cold pad winter hours cover hot
 2 food eat eating product picky feed love difference dry diet
 3 ferret ferrets like getting use little house gross just good
 4 product water using just works conditioner use good great junk
 5 food water just pain product make day eat don days
 6 flexible like ve pug held air airline does just years
 7 pup dog easy little puppy treats hip miniature dachshund bite
 8 product package buy money work disappointed products like ordered cleans
 9 filter media filters aquaclear filtration use bio carbon great hob
10 cut nails dog nail dogs use like cutting sharp easy
11 nail clippers quick nails cut use work sharp easy just
12 milk just dog treat like don think rubber say really
13 vitamin meal dried min max protein fat crude calcium vitamins
14 fleas dog product flea dogs advantage works use ticks frontline
15 fish tank water product bottle use week high ve ml
16 cats inexpe

In this case, the topics are reasonably coherent, so I'm going to move on.

If things look messy:
 - n_topics might be too large, either for the diversity in the corpus (maybe there really aren't 1000 topics), or for the number of documents you have (you just don't have enough data)
 - n_topics might be too low (real topics have to be merged together by the algorithm, which doesn't work well)
 - you've got a bug!
 
Setting n_topics very small (say 5) or very high (say 1000) is a good way of building up some intuition for what works (although beware `n_topics=1000` will take a long time to run).

# Inspect a document in the corpus

The topic model is the lens through which we're going to view future documents.

But let's first look at our existing documents through this lens.

To do that we have to transform the documents we trained on to be distributions of topics (e.g. document 1 is 20% topic A, 30% topic B, etc.)

We do that by running the `lda.transform` method on the vectorized documents `X`:

In [17]:
doc_topic = lda.transform(X)

`doc_topic` has a row for each document, and a column for each topic.

In [18]:
doc_topic.shape

(20000, 100)

Finally let's look at the topic distribution of a random document

In [19]:
texts[1234]

"Although I didn't order this from Amazon, I highly highly highly recommend this product. I get mine from my local pet store (when I get a coupon) but spent a lot of time here reading reviews. I did months of research on different supplements, allergy relief, and all in one solution for my dogs health. I wanted him on Vitamin C to prevent hip problems later on, wanted something to give a healthy coat (not salmon oil) and wanted something natural and made in the USA. I can understand the ingredients and the product has been around for years. I am very pleased with it, my dogs allergies are under control, his coat his healthy and shiny, and he has never ending amounts of energy. He feels great, looks great, and acts great. When you add water it makes a great gravy for their food and he spends a lot of time licking the bowl CLEAN. Don't waste time/money/effort looking at other products, getting crap you see at Petsmart, buy NuPro and let it work. You wont be disappointed."

In [20]:
top_topics = (doc_topic[1234]).argsort()[:-6:-1]
print(top_topics)

[23 49 64  8 77]


What are these topics?

In [21]:
for i in top_topics:
    print_topic(topic_term, i, vocabulary)

23 food cat cats eat foods dry like chicken brands diet
49 dogs oil love dog skin product salmon coat coats small
64 price great good quality product amazon better value best size
 8 product package buy money work disappointed products like ordered cleans
77 pet store amazon local product price stores cheaper buy shipping


# Visualization

pyLDAvis is a comprehensive package for visualizing the results of a topic model. It's useful for understanding the structure of the model you've just discovered. The topics exist in a huge space. This package squeezes things down to 2D so we can look at it on the screen.

In my experience, it generates a ton of spurious warnings, so let's disable warnings for this package when we import it (a useful trick!)

In [22]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    try:
        import pyLDAvis
    except ImportError:
        print('ERROR: pyLDAvis not installed! Skip to next section!')

In addition to the `topic_term` and `doc_topic` matrices, pyLDAvis needs to know how often each word occurs in the entire corpus, and how long each document is. Here are calculations that give those.

In [23]:
term_frequency = np.asarray(X.sum(axis=0)).squeeze()
doc_lengths = [len(t) for t in texts]

In [24]:
lda_vis = pyLDAvis.prepare(topic_term_dists=topic_term,
                           doc_topic_dists=doc_topic,
                           doc_lengths=doc_lengths,
                           vocab=vocabulary,
                           term_frequency=term_frequency)

In [25]:
pyLDAvis.display(lda_vis)

# Put all this together in a Pipeline and persist the model

The process of getting from document to topic distribution is a little fiddly. We need to:
 - Vectorize the document (using the same vocabulary we used when training above)
 - Transform the document using the LDA object
 
scikit-learn allows us to bundle these steps (and more!) together in an object called a `Pipeline`, which we can save to disk, reload, and work with again. Let's build one, train it, and save it.

**WARNING**: this next cell will take a while to execute the first time you run it. After that though, the model will be loaded from disk.

In [26]:
import pickle
from sklearn.pipeline import make_pipeline

try:
    with open('topic_model.pkl', 'rb') as f:
        topic_pipeline = pickle.load(f)
    pipeline_vocabulary = topic_pipeline.steps[0][1].get_feature_names()
except IOError:
    topic_pipeline = make_pipeline(
        CountVectorizer(stop_words='english', max_features=10000),
        LatentDirichletAllocation(n_topics=100, learning_method='batch', n_jobs=-2, random_state=0)
    )
    topic_pipeline.fit(texts)
    with open('topic_model.pkl', 'wb') as f:
        pickle.dump(topic_pipeline, f)
        
pipeline_vocabulary = topic_pipeline.steps[0][1].get_feature_names()
pipeline_topic_term = topic_pipeline.steps[1][1].components_

# Determine topics of a new document

The single document we looks at above was a pretty short document. Let's make a more interesting, longer document out of all the reviews of that product.

In [27]:
randomreviews = " ".join(texts[5000:5005])

In [28]:
print(randomreviews)

This does an amazing job helping catch tons of stuff the first day it was already full of stuff only problem is when I saw the box the charcoal residue was all over the box they do not put them in a bag as all the other filters we purchase however no big deal! Would definitely reccomend These are just the right size and are clean when they arrive in the package, unlike some that will sometimes have charcoal loose in the package. If you have a five gallon hex, this is the only filter selection you get. The length of effectiveness depends on the condition of your tank, number of fish you keep, etc.  If you have a light tank load, you can rinse these at least one time and reuse.  I am able to rinse after two weeks and replace after roughly four to six weeks.  Use it much beyond that and you just make your maintenance and water changes more of a workload.  As a point of reference I am doing partial water changes two or three times a week and am able to keep very clear water.  Marineland is

In [29]:
doc_topic = topic_pipeline.transform([randomreviews])

In [30]:
print(doc_topic.shape)

(1, 100)


In [31]:
top_topics = (doc_topic[0]).argsort()[:-6:-1]
print(top_topics)

[29 77 79 39 17]


In [32]:
for i in top_topics:
    print_topic(pipeline_topic_term, i, pipeline_vocabulary)

29 water filter tank carbon gallon clear change fish filters aquarium
77 pet store amazon local product price stores cheaper buy shipping
79 water fish tank use clear tanks gallon changes great clean
39 test kit easy ammonia tests use results accurate water instructions
17 stuff great works use just turtle love easy smell product


# What next?

Use the `doc_topic` array for a downstream task, e.g.
 - corpus exploration (remember the [NYT visualization](http://christo.cs.umass.edu/NYT/))
 - document clustering, e.g. use something like `KMeans` (in scikit-learn) to visualize which documents are most similar in terms of their topics, which may surface groups of topics or groups of documents

## Summarization

Here's a short algorithm, but see [Fast Forward Labs Report 04](http://ff04.fastforwardlabs.com) for details:
  - Train LDA on all products of a certain type (e.g. all the books)
  - Treat all the reviews of a particular product as one document, and infer their topic distribution
  - Infer the topic distribution for each sentence
  - For each topic that dominates the reviews of a product, pick some sentences that are themselves dominated by that topic

<img src="img/strain.png">

Be aware of limitations:
 - Choosing `n_topics` is an art rather than a science!
 - The topics don't come with names. Sometimes they overlap. Sometimes they're not what you want them to be. For example, if you run a topic model on the NYT corpus, there's no guarantee you'll get topics that correspond to the sections of the newspaper (business, metro, world, sport, etc.!)

## Further reading

The way you used `fit` and `transform` for both the `vectorizer`, `lda`, and `topic_pipeline` objects is generic across scikit-learn, so play with scikit-learn, e.g. [Andreas Mueller's presentation](https://www.youtube.com/watch?v=8CzwlZbwDkI) is a good place to start.
 
Remember if you're interested in the LDA algorithm itself, take a look at

 - [Tim Hopper's PyData NYC 2015 talk](https://www.youtube.com/watch?v=_R66X_udxZQ)
 - [David Blei's ACM article](https://www.cs.princeton.edu/~blei/papers/Blei2012.pdf)
 
[Fast Forward Labs Report 4](http://ff04.fastforwardlabs.com/) goes into more detail on the summarization use case in particular, and also talks about the latest and greatest approach: recurrent neural networks and neural language embeddings. You may find this [PyGotham talk](https://www.youtube.com/watch?v=y7XoypvQRhY&feature=youtu.be) a useful semi-technical alternative to the report.

And our demo of [Luhn's Algorithm](http://www.fastforwardlabs.com/luhn/) is a fun and concrete example of the simplest possible thing you could do.